In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

## 3D meshing example

This notebook shows how to mesh a 3D volume:

1. Load and visualize a volume
1. Apply image filters and segment image
1. Generate a 3D surface mesh from the binary image
1. Decimate/simplify a 3D mesh
1. Visualize and save the mesh to gmsh22 format

## Todo:

- [ ] Mimic `add_points` from 2d meshing submodule
- [ ] Tune performance
- [ ] Add edge points instead of padding?

In [ ]:
from nanomesh.volume import Volume
import pyvista as pv
from skimage import filters
import numpy as np

Load the data and select a subvolume to work with for this notebook.

In [ ]:
full_vol = Volume.load('slab_x450-550.npy')
vol = full_vol.select_subvolume(zs=(710, 790), ys=(320,360))
vol.show_slice()

In [ ]:
vol_gauss = vol.apply(filters.gaussian, sigma=5)

thresh = vol_gauss.apply(filters.threshold_li)

seg_vol = vol_gauss.apply(np.digitize, bins=[thresh])

seg_vol.show_slice()

In [ ]:
from nanomesh.mesh2d import get_edge_coords, pairwise_circle
from sklearn.cluster import KMeans
from nanomesh._mesh_shared import add_points_kmeans


def generate_interface_coords(image, label=1):
    edges = image[0], image[-1], image[:,0], image[:,-1], image[:,:,0], image[:,:,-1]

    x,y,z = image.shape
    shifts = (
        (0,0),
        (0,x-1),
        (1,0),
        (1,y-1),
        (2,0),
        (2,z-1),
    )

    all_points = []
    for edge, (index, coord) in zip(edges, shifts):
        points = add_points_kmeans(edge, label=label)
        points = np.insert(points, index, values=coord, axis=1)
        all_points.append(points)
    
    return np.vstack(all_points)


def generate_corner_points(image,
                           point_density = 1/10,
                           label = 1):
    x, y, z = image.shape

    corners = {
        1: (0,0,0),
        2: (0,0,z-1),
        3: (0,y-1,0),
        4: (0,y-1,z-1),
        5: (x-1,0,0),
        6: (x-1,0,z-1),
        7: (x-1,y-1,0),
        8: (x-1,y-1,z-1),
    }

    edges = (
        (1,2),
        (1,3),
        (1,5),
        (2,4),
        (2,6),
        (3,4),
        (3,7),
        (5,6),
        (5,7),
        (8,4),
        (8,6),
        (8,7),
    )

    corner_points = [np.array(list(corners.values()))]

    for i,j in edges:
        ci = np.array(corners[i])
        cj = np.array(corners[j])

        index = np.linspace(ci, cj, int(l)+1, dtype=int)[1:-1]
        seg = image[index[:,0],index[:,1],index[:,2]]

        n_points = int(np.sum(seg == label) * point_density)

        if n_points:
            kmeans = KMeans(n_clusters=n_points).fit(index)
            corner_points.append(kmeans.cluster_centers_)
    
    return np.vstack(corner_points)


interface_points = generate_interface_coords(seg_vol.image)
print(interface_points.shape)
print(interface_points.min(axis=0), interface_points.max(axis=0))

corner_points = generate_corner_points(seg_vol.image)
print(corner_points.shape)
print(corner_points.min(axis=0), corner_points.max(axis=0))

### Generate 3d tetragonal mesh

Meshes can be generated using the `Mesher3D` class.

A convenience method is also available through `volume.generate_mesh`, which offers less flexibility.

In [ ]:
%%time

from nanomesh.mesh3d import Mesher3D

mesher = Mesher3D(seg_vol.image)
# mesher.pad(pad_width=2)
mesher.generate_contour(label=0)
mesher.generate_contour(label=1)

mesher.add_points(point_density=1/500, label=1, verbose=3, max_iter=1)

mesher.generate_surface_mesh(step_size=1)
mesher.points[1].append(corner_points)
mesher.points[1].append(interface_points)
mesher.smooth_mesh()
mesher.simplify_mesh(n_faces=5000)
mesher.generate_volume_mesh()
mesher.generate_domain_mask()
mesh = mesher.to_meshio()

pv.plot_itk(mesh)

Save the data.

In [ ]:
mesh.write("volume.msh", file_format='gmsh22', binary=False)